<a href="https://colab.research.google.com/github/ZeddBR/projeto-aula/blob/main/Music_Updater.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
%pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [7]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")

In [8]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""

    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [9]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="agente que busca informacoes no Google",
        tools=[google_search],
        instruction="""
        Voce é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para realizar uma busca geral da história, novidades e últimos lançamentos musicais de bandas ou artistas da música mundial.
        Buscar novidades e noticias sobre as bandas e citar singles ou clipes lançados na última semana ou de uma banda especificada no topico.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


In [50]:
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Voce é um redator de conteudo musical, especialista em músicas e bandas de rock do estilo emocore, emo, post-emo, midwest emo.
        Com base na lista de lancamentos mais recentes e relevantes buscados, voce deve:
        usar a ferramenta de busca do google (google_search) para criar uma lista detalhada, e atual,
        listando novidades, nome da faixa, artista, data de lançamento, nome do álbum se houver conforme informacoes recebidas do agente anterior.
        Liste também as redes sociais do artista e as plataformas disponíveis para escutar a música ou assistir ao clipe com links de redirecionamento.
        Voce tambem pode usar o (google_search) para encontrar informacoes sobre os temas e aprofundar.
        """,
        description="Agente que gera a lista e conteudos",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    lista_top_10 = call_agent(planejador, entrada_do_agente_planejador)
    return lista_top_10

In [55]:
def agente_redator(topico, lista_top_10):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado listar de forma organizada o conteudo gerado pelo agente planejador, sem repetir informações.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado. Verifique se os links estão funcionais antes de envia-los.
            Gere a lista sobre o tópico indicado, liste organizando de forma clara nome do artista do topico, da música ou faixa, album, ano de lançamento e links
            das redes sociais e das plataformas de streaming disponíveis.
            Utilize fonte em negrito para títulos. Se possível, inclua uma foto da capa do album ou do artista/banda.
            Apenas liste, sem comentários adicionais, de forma bem organizada e resumida com os links, agenda de shows a partir de hoje e uma lista de até 10 músicas mais escutadas do artista do topico.
            """,
        description="Agente redator e do post final."

    )

    entrada_do_agente_redator = f"Tópico: {topico}\nRascunho: {lista_top_10}"
    lista_final = call_agent(redator, entrada_do_agente_redator)
    return lista_final

In [57]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🎧 Buscando novidades do seu artista favorito para você! 🎧")

topico = input("❓ Por favor, digite o nome do artista ou banda que deseja receber as últimas atualizações: ")

if not topico:
  print("Voce esqueceu de digitar o nome do artista/banda")
else:
    print(f"Pronto! Agora se liga nas novidades de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- Busca Principal ---\n")
    display(to_markdown(lancamentos_buscados))
    print("-------------------------------------------------------")

    lista_top_10 = agente_planejador(topico, lancamentos_buscados)
    print("\n--- Novidades! ---\n")
    display(to_markdown(lista_top_10))
    print("-------------------------------------------------------")

    lista_final = agente_redator(topico, lista_top_10)
    print("\n--- Links e atualizações ---\n")
    display(to_markdown(lista_final))
    print("-------------------------------------------------------")

🎧 Buscando novidades do seu artista favorito para você! 🎧
❓ Por favor, digite o nome do artista ou banda que deseja receber as últimas atualizações: Bullet for my valentine
Pronto! Agora se liga nas novidades de Bullet for my valentine

--- Busca Principal ---



> Olá! Para fornecer as últimas informações sobre o Bullet For My Valentine, incluindo novidades, lançamentos recentes e outras notícias relevantes, vou realizar algumas pesquisas no Google.
> 
> 
> Aqui estão as últimas notícias sobre o Bullet For My Valentine:
> 
> *   **Fim da turnê com o Trivium:** O Bullet For My Valentine confirmou o fim da turnê "Poisoned Ascendancy" com o Trivium. A turnê celebrava os 20 anos dos álbuns "The Poison" do Bullet For My Valentine e "Ascendancy" do Trivium. A etapa norte-americana da turnê foi encerrada em 18 de maio de 2025. Havia planos de continuar a turnê pela América do Sul e Austrália, mas esses planos foram cancelados.
> *   **Declarações sobre o fim da turnê:** O cancelamento das datas da turnê gerou controvérsia, com o baixista do Trivium, Paolo Gregoletto, culpando o vocalista do Bullet For My Valentine, Matt Tuck, pela decisão. O Bullet For My Valentine lançou uma declaração agradecendo aos fãs e afirmando que a banda está focando no "próximo capítulo". Matt Heafy, vocalista do Trivium, pediu paz após a polêmica.
> *   **Novo álbum:** O Bullet For My Valentine planeja entrar em estúdio no verão de 2025 para trabalhar em um novo álbum, que eles prometem ser o melhor até agora. Matt Tuck mencionou que a banda está trabalhando em 13 faixas para o oitavo álbum de estúdio. Eles também estão planejando turnês para 2026 e 2027.
> 
> Lançamentos musicais recentes e singles:
> 
> *   O álbum mais recente do Bullet For My Valentine é o álbum auto-intitulado "Bullet For My Valentine", lançado em 5 de novembro de 2021. Uma edição deluxe do álbum foi lançada em agosto de 2022.
> *   **Singles recentes:**
>     *   "Halo (Urbandawn Remix)" (2023)
>     *   "Halo (Matt Tuck Rework)" (2023)
>     *    "Omen" (2022)
>     *   "No More Tears To Cry" (2022)
>     *   "Stitches" (2022)
> *   A banda lançou uma edição de 20º aniversário do álbum "The Poison" em 29 de novembro de 2024, incluindo faixas ao vivo gravadas em Brixton.
> 
> Espero que esta informação seja útil!
> 


-------------------------------------------------------

--- Novidades! ---



> Com prazer! Aqui está uma lista detalhada e atualizada sobre o Bullet For My Valentine, com base nas informações fornecidas e em pesquisas adicionais:
> 
> **Bullet For My Valentine: Novidades e Lançamentos Recentes**
> 
> **Notícias:**
> 
> *   **Fim da Turnê "Poisoned Ascendancy" com Trivium:** A turnê que celebrava os 20 anos dos álbuns "The Poison" (BFMV) e "Ascendancy" (Trivium) foi finalizada. A etapa norte-americana encerrou-se em 18 de maio de 2025. As datas planejadas para a América do Sul e Austrália foram canceladas.
> *   **Controvérsia e Declarações:** O cancelamento gerou discussões, com Paolo Gregoletto (Trivium) mencionando Matt Tuck (BFMV) como responsável. O Bullet For My Valentine agradeceu aos fãs e anunciou foco no "próximo capítulo". Matt Heafy (Trivium) buscou apaziguar a situação.
> *   **Novo Álbum:** A banda planeja entrar em estúdio no verão de 2025 para trabalhar em um novo álbum, com previsão de ser o melhor trabalho deles. Matt Tuck informou que estão trabalhando em 13 faixas para o oitavo álbum de estúdio. Turnês estão sendo planejadas para 2026 e 2027.
> 
> **Lançamentos Musicais Recentes e Singles:**
> 
> *   **Álbum Mais Recente:** "Bullet For My Valentine" (auto-intitulado), lançado em 5 de novembro de 2021. A edição deluxe saiu em agosto de 2022.
> *   **Singles Recentes:**
>     *   "Halo (Urbandawn Remix)" (2023)
>     *   "Halo (Matt Tuck Rework)" (2023)
>     *   "Omen" (2022)
>     *   "No More Tears To Cry" (2022)
>     *   "Stitches" (2022)
> *   **The Poison - 20th Anniversary Edition:** Lançada em 29 de novembro de 2024, com gravações ao vivo de Brixton.
> 
> Para ficar por dentro de todas as novidades, você pode seguir o Bullet For My Valentine nas redes sociais:
> 
> *   **Website:** [https://www.bulletformyvalentine.com/](https://www.bulletformyvalentine.com/)
> *   **Facebook:** [https://www.facebook.com/BulletForMyValentine](https://www.facebook.com/BulletForMyValentine)
> *   **X:** [https://twitter.com/bfmvofficial](https://twitter.com/bfmvofficial)
> *   **Instagram:** [https://www.instagram.com/bfmvofficial/](https://www.instagram.com/bfmvofficial/)
> 
> Você pode ouvir os singles e álbuns recentes do Bullet For My Valentine nas seguintes plataformas:
> 
> *   **Spotify:** [https://open.spotify.com/artist/7iWiAD5LLKyiox2MGpjdwQ](https://open.spotify.com/artist/7iWiAD5LLKyiox2MGpjdwQ)
> *   **YouTube:** [https://www.youtube.com/user/BFMVofficial](https://www.youtube.com/user/BFMVofficial)
> *   **Apple Music:** procure por Bullet For My Valentine na Apple Music
> *   **Amazon Music:** procure por Bullet For My Valentine na Amazon Music
> 


-------------------------------------------------------

--- Links e atualizações ---



> Com certeza! Preparei um resumo organizado sobre o Bullet For My Valentine, perfeito para você ficar por dentro de tudo!
> 
> ### **Bullet For My Valentine: Tudo Que Você Precisa Saber**
> 
> #### **Notícias:**
> 
> *   **Novo Álbum:** A banda planeja entrar em estúdio no verão de 2025 para trabalhar em um novo álbum, com previsão de lançamento em 2026 ou 2027.
> *   **Turnê:** As datas planejadas para a América do Sul e Austrália foram canceladas, mas turnês estão sendo planejadas para 2026 e 2027 para promover o novo álbum.
> 
> #### **Lançamentos Musicais Recentes e Álbuns:**
> 
> *   **Bullet For My Valentine (álbum)**
> 
>     *   **Lançamento:** 5 de novembro de 2021 (edição deluxe em agosto de 2022)
> 
> *   **The Poison - 20th Anniversary Edition**
> 
>     *   **Lançamento:** 29 de novembro de 2024 (gravações ao vivo de Brixton)
> 
> #### **Singles Recentes:**
> 
> *   Halo (Urbandawn Remix) - (2023)
> *   Halo (Matt Tuck Rework) - (2023)
> *   Omen - (2022)
> *   No More Tears To Cry - (2022)
> *   Stitches - (2022)
> 
> #### **Redes Sociais e Plataformas de Streaming:**
> 
> *   **Website:** [https://www.bulletformyvalentine.com/](https://www.bulletformyvalentine.com/)
> *   **Facebook:** [https://www.facebook.com/BulletForMyValentine](https://www.facebook.com/BulletForMyValentine)
> *   **X:** [https://twitter.com/bfmvofficial](https://twitter.com/bfmvofficial)
> *   **Instagram:** [https://www.instagram.com/bfmvofficial/](https://www.instagram.com/bfmvofficial/)
> *   **Spotify:** [https://open.spotify.com/artist/7iWiAD5LLKyiox2MGpjdwQ](https://open.spotify.com/artist/7iWiAD5LLKyiox2MGpjdwQ)
> *   **YouTube:** [https://www.youtube.com/user/BFMVofficial](https://www.youtube.com/user/BFMVofficial)
> *   **Apple Music:** procure por Bullet For My Valentine na Apple Music
> *   **Amazon Music:** procure por Bullet For My Valentine na Amazon Music
> 
> #### **Shows:**
> 
> *   Não há shows agendados no momento.
> 
> #### **Top 10 Músicas Mais Escutadas:**
> 
> 1.  Tears Don't Fall
> 2.  Waking The Demon
> 3.  Scream Aim Fire
> 4.  Your Betrayal
> 5.  All These Things I Hate (Revolve Around Me)
> 6.  Hearts Burst Into Fire
> 7.  The Last Fight
> 8.  Hand Of Blood
> 9.  Knives
> 10. Rainbow Veins
> 


-------------------------------------------------------
